In [19]:
# imoprt libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,cross_validate,cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import log_loss,classification_report,confusion_matrix, roc_curve, roc_auc_score,accuracy_score
from mlxtend.classifier import StackingClassifier,EnsembleVoteClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC 
from xgboost.sklearn import XGBClassifier

import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline

from collections import Counter

# calculate weights for imbablanced classes
from sklearn.utils import class_weight


In [2]:
df = pd.read_csv("../data/train_norm.csv")
df.head()

sss = StratifiedShuffleSplit(n_splits=1,test_size=0.1, random_state=42)
print(sss)

X = df.drop("target",axis=1)
y = df.target

train_index = []
test_index = []

for tr, tes in sss.split(X,y):
    print("TRAIN:", tr, "TEST:", tes)
    train_index = tr
    test_index = tes

X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

print("Shapes of data sets")
print("X_train: ", X_train.shape, "y_train: ", y_train.shape)
print("X_train: ", X_test.shape,"y_test: ", y_test.shape)

StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.1,
            train_size=None)
TRAIN: [57972 30244  9427 ..., 60232 28576 27516] TEST: [59081 21681 51999 ...,  1777   269 53901]
Shapes of data sets
X_train:  (55690, 93) y_train:  (55690,)
X_train:  (6188, 93) y_test:  (6188,)


In [8]:
clf1 = KNeighborsClassifier(n_neighbors=5)
clf2 = LogisticRegression()
clf3 = GaussianNB()
clf4 = SVC()
clf5 = RandomForestClassifier(random_state=42)
clf6 = GradientBoostingClassifier(n_estimators=100)
clf7 = XGBClassifier()

lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf4, clf2, clf3], 
                          meta_classifier=lr)

sclf2 = StackingClassifier(classifiers=[clf1, clf2, clf5],meta_classifier=lr)

#print('3-fold cross validation:\n')
#scores = cross_val_score(sclf, X_train, y_train, cv=3, scoring='neg_log_loss')
                          
# for clf, label in zip([clf1, clf2, clf3, sclf], 
#                       ['KNN', 
#                        'Logistic Regression', 
#                        'Naive Bayes',
#                        'StackingClassifier']):

#     scores = cross_val_score(clf, X_train, y_train, 
#                                               cv=3, scoring='neg_log_loss')
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
#           % (scores.mean(), scores.std(), label))

In [15]:
scores

array([-1.37907159, -1.38520586, -1.39405126])

In [4]:
sclf.fit(X_train,y_train)

StackingClassifier(average_probas=False,
          classifiers=[SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), LogisticRegression(C=1.0, class_weight=None, dual=False, fit_...ne, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), GaussianNB(priors=None)],
          meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [5]:
sclf.score(X_test,y_test)

0.57514544279250157

In [7]:
log_loss(y_test,sclf.predict_proba(X_test))

1.342947925536363

In [9]:
sclf2.fit(X_train,y_train)

StackingClassifier(average_probas=False,
          classifiers=[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'), LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, ...estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)],
          meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [10]:
sclf2.score(X_test,y_test)

0.69198448610213315

In [12]:
log_loss(y_test,sclf2.predict_proba(X_test))

1.8943833009244573

In [14]:
lr = LogisticRegression(random_state=42)
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(n_estimators=1000,random_state=42)

lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [16]:
gb.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)

In [17]:
gb.score(X_test,y_test)

0.81237879767291532

In [39]:
log_loss(y_test,gb.predict_proba(X_test))

0.49755135352409918

In [30]:
eclf = EnsembleVoteClassifier(clfs=[lr, rf, gb], weights=[1,1,2], refit=False)

eclf.fit(X_test, y_test)



EnsembleVoteClassifier(clfs=[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), RandomForestClassifier(bootstr...         presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)],
            refit=False, verbose=0, voting='hard', weights=[1, 1, 2])

In [31]:
eclf.score(X_test,y_test)

0.80575307045895284

In [32]:
log_loss(y_test,eclf.predict_proba(X_test))

0.51784678477965185

In [33]:
lr.predict(X_test[:15]), rf.predict(X_test[:15]), gb.predict(X_test[:15]), eclf.predict(X_test[:15]), y_test[:15].values

(array([9, 2, 8, 5, 8, 3, 6, 3, 8, 2, 9, 9, 2, 3, 9]),
 array([6, 3, 8, 5, 8, 3, 6, 3, 8, 2, 9, 9, 2, 2, 9]),
 array([9, 3, 8, 5, 8, 3, 6, 3, 8, 2, 8, 9, 2, 2, 9]),
 array([9, 3, 8, 5, 8, 3, 6, 3, 8, 2, 8, 9, 2, 2, 9]),
 array([9, 3, 8, 5, 7, 3, 6, 3, 8, 2, 8, 9, 2, 3, 9]))

In [38]:
np.round(lr.predict_proba(X_test[:15]),2), np.round(rf.predict_proba(X_test[:15]),2), \
                np.round(gb.predict_proba(X_test[:15]),2), np.round(eclf.predict_proba(X_test[:15]),2), y_test[:15].values

(array([[ 0.15,  0.07,  0.16,  0.  ,  0.  ,  0.13,  0.07,  0.16,  0.26],
        [ 0.  ,  0.64,  0.21,  0.09,  0.01,  0.01,  0.02,  0.01,  0.01],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.01,  0.99,  0.  ],
        [ 0.  ,  0.01,  0.01,  0.  ,  0.98,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.01,  0.  ,  0.  ,  0.01,  0.25,  0.72,  0.  ],
        [ 0.  ,  0.24,  0.52,  0.23,  0.  ,  0.  ,  0.01,  0.  ,  0.  ],
        [ 0.05,  0.02,  0.  ,  0.01,  0.  ,  0.88,  0.02,  0.01,  0.01],
        [ 0.  ,  0.42,  0.56,  0.  ,  0.01,  0.  ,  0.  ,  0.01,  0.  ],
        [ 0.04,  0.  ,  0.  ,  0.  ,  0.  ,  0.01,  0.  ,  0.94,  0.02],
        [ 0.  ,  0.65,  0.24,  0.05,  0.  ,  0.01,  0.03,  0.  ,  0.01],
        [ 0.11,  0.15,  0.07,  0.  ,  0.  ,  0.01,  0.03,  0.25,  0.37],
        [ 0.01,  0.1 ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.06,  0.83],
        [ 0.  ,  0.86,  0.04,  0.1 ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.44,  0.48,  0.  ,  0.  ,  0.01, 